# CUDA-Python with Numba

### 2D Threadblocks

CUDA allows us to use 1, 2, or 3 dimensional grids and threadblocks.  In this example, we'll utilize a two-dimensional threadblock to simplify our kernel definition.

The kernel that we'll write computes the distance between all pairs of atoms within single molecules. That is, we'll compute intramolecular distances for each molecule in a list of molecules.  

In [1]:
import numba
from numba import cuda
import numpy as np
import math

### Input Data

For a list of `N` molecules with at most `ATOM_MAX` atoms, each of which has `3` spatial coordinates specified, our input data will be shaped `(N, ATOM_MAX, 3)`.  We'll generate some random coordinates and store these in the array `crds`.

The result array will be shaped `(N, ATOM_MAX, ATOM_MAX)`.

In [2]:
# CONSTANTS
ATOM_MAX = 29
N = int(1e4)

In [3]:
crds = np.random.random((N, ATOM_MAX, 3))
crds_natoms = np.array([ATOM_MAX]*N)

# create result array on the GPU
shape = (N, ATOM_MAX, ATOM_MAX)
distance_mat_gpu = cuda.device_array(shape, dtype=np.float32)

### Kernel Design

We'll leverage the block-thread hierarchy in CUDA to make our kernel design simple.  Specifically, we'll use a 2-D threadblock such that each thread will compute a distance between a unique pair of atoms in our array.  In other words, `threadIdx.x` and `threadIdx.y` will index into atoms `i` and `j` of each molecule.  It follows that `blockIdx.x` should be 1-D and index over molecules.

In [4]:
@cuda.jit
def compute_distance_mat(crds, natoms, dmat):
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    
    dmat[bx, tx, ty] = 0.0
    
    # bounds check
    N = natoms[bx]
    if tx >= N: return
    if ty >= N: return 
    
    for i in range(3):
        dmat[bx, tx, ty] += ( crds[bx, tx, i] - crds[bx, ty, i] )**2
    dmat[bx, tx, ty] = math.sqrt(dmat[bx, tx, ty])  

### Call CUDA Kernel
First, we'll want to move our data to the GPU.  We only have to do this for the `crds` array.  The results array doesn't exist, so we'll create space for this on the GPU.

Next, we want to launch the kernel.  To do so, we'll have to include a launch configuration that specifies how many blocks and threads to use.  As we discussed above, we want this kernel to use a 2-D threadblock so that `threadIdx.x` and `threadIdx.y` index into atoms `i` and `j` of each molecule.  It follows that `blockIdx.x` should be 1-D and index over molecules.

In [5]:
%%timeit -n1 -r1 -o

#transfer structures data (the coordinates) to GPU
crds_gpu = cuda.to_device(crds)
crds_natoms_gpu = cuda.to_device(crds_natoms)

# launch kernel
compute_distance_mat[ N , (29, 29)](crds_gpu, crds_natoms_gpu, distance_mat_gpu)

# copy the data back
distance_mat_cpu = distance_mat_gpu.copy_to_host()

210 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 210 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [6]:
# store the timing result
GPU_TIMING = _

### Naive Numpy Version
There are better ways to do this calculation, but they require reshapes and broadcasts.  For the sake of simplicity and ensuring that we're computing the correct result, we use loops.

In [7]:
dmat_np_cpu = np.zeros((N, ATOM_MAX, ATOM_MAX))

In [8]:
%%timeit -n1 -r1 -o

for m in range(N):
    for i in range(ATOM_MAX):
        for j in range(ATOM_MAX):
            dmat_np_cpu[m,i,j] = np.sqrt(np.sum((crds[m, i] - crds[m,j])**2))

56.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 56.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [9]:
# store the timing result
CPU_TIMING = _

### Speedup Factor

In [10]:
print('Speedup factor: ', CPU_TIMING.average / GPU_TIMING.average, 'X')

Speedup factor:  269.10209998987614 X


### Checking Results

In [11]:
# copy the data back (again)
distance_mat_cpu = distance_mat_gpu.copy_to_host()

# check results
tol = 1e-4
error = 0
for m in range(N):
    for i in range(ATOM_MAX):
        for j in range(ATOM_MAX):
            r = dmat_np_cpu[m,i,j] - distance_mat_cpu[m,i,j]
            if r>tol:
                error=1
if not error:
    print('results agree')

results agree
